In [8]:
import yfinance as yf, pandas as pd, numpy as np

'''
Calculating Fundamental Trading Metrics:
- Sharpe Ratio
- 2024 Market Cap
- Revenue Growth Rate
- Price X Revenue
- Gross Profit Margins
- Buyback Value
'''

tickers = [
    'META', 'NICE', 'KLAC', 'REGN', 'MITK', 'AMAT', 'FIS', 'COUR',
    'GILD', 'GOOG', 'UIS', 'AMD', 'ROK', 'DELL', 'GNRC', 'TXN',
    'BP', 'ILMN', 'WFC', 'NOK', 'RMBS', 'SNPS', 'EXAS', 'CCL',
    'PARA', 'CRWD', 'PYPL', 'INTC'
]

# Calculate Sharpe Ratio for One Stock
def get_sharpe_ratio(ticker):
    equity = yf.Ticker(ticker)
    hist = equity.history(period='1y')
    returns = hist['Close'].pct_change().dropna()

    excess_returns = returns - .0432 / 252 # Using 3-month TBill as Risk-Free Rate
    sharpe = excess_returns.mean() / excess_returns.std() * np.sqrt(252) # 252 trading days

    return round(sharpe, 3)

# Calculate Sharpe Ratio for All Stocks
def all_sharpe_ratios(tickers):
    metrics = {}

    for ticker in tickers:
        metrics[ticker] = get_sharpe_ratio(ticker)

    return metrics

# 2024 End Of Year Market Caps
def get_dec31_2024_market_cap(ticker):

    stock = yf.Ticker(ticker)

    start_date = '2024-12-31'
    end_date = '2025-01-02'

    hist = stock.history(start=start_date, end=end_date)

    # Last trading day of 2024 (Dec 31, 2024)
    last_price_2024 = hist['Close'].iloc[0]  # First available price
    price_date = hist.index[0].strftime('%Y-%m-%d')

    # Get current shares outstanding
    info = stock.info
    shares_outstanding = info.get('sharesOutstanding', 0)

    # Calculate market cap in billions
    market_cap_billions = (last_price_2024 * shares_outstanding) / 1e9

    return market_cap_billions

def get_all_eoy_market_caps(tickers):
    metrics = {}
    for ticker in tickers:
        metrics[ticker] = round(get_dec31_2024_market_cap(ticker), 2)
    return metrics

# Get Revenue Growth from Financial Statements
def get_revenue_growth(ticker):
    stock = yf.Ticker(ticker)

    # Try getting from info first
    info = stock.info
    revenue_growth = info.get('revenueGrowth')
    if revenue_growth is not None:
        return round(revenue_growth * 100, 2)

    # Calculate from financials if info doesn't have it
    financials = stock.financials
    if not financials.empty and 'Total Revenue' in financials.index:
        revenues = financials.loc['Total Revenue'].dropna()
        if len(revenues) >= 2:
            latest_revenue = revenues.iloc[0]  # Most recent year
            previous_revenue = revenues.iloc[1]  # Previous year
            growth = ((latest_revenue - previous_revenue) / previous_revenue) * 100
            return round(growth, 2)

    return 0

# All Revenue Growth Rates
def get_all_revenue_growth(tickers):
    metrics = {}
    for ticker in tickers:
        metrics[ticker] = get_revenue_growth(ticker)
    return metrics

# Price X Revenue (P/S Ratio)
def get_price_x_revenue(ticker):
    stock = yf.Ticker(ticker)

    # Try getting from info first
    pxr = stock.info.get('priceToSalesTrailing12Months')
    if pxr is not None:
        return round(pxr, 2)

    # If no info, manual calculation: Market Cap / Annual Revenue
    market_cap = stock.info.get('marketCap', 0)

    # Get revenue from financials
    financials = stock.financials
    if not financials.empty and 'Total Revenue' in financials.index and market_cap > 0:
        annual_revenue = financials.loc['Total Revenue'].iloc[0]  # Most recent year
        pxr_calculated = market_cap / annual_revenue
        return round(pxr_calculated, 2)

    return 0

# All PxR
def get_all_pxr(tickers):
    measures = {}
    for ticker in tickers:
        measures[ticker] = get_price_x_revenue(ticker)
    return measures

# Get Gross Profit Margins
def get_gross_profit_margins(ticker):
    stock = yf.Ticker(ticker)

    # Try getting from info first
    gross_margin = stock.info.get('grossMargins')
    if gross_margin is not None:
        return round(gross_margin * 100, 2)

    # Calculate from financial statements
    financials = stock.financials
    if not financials.empty:
        try:
            total_revenue = financials.loc['Total Revenue'].iloc[0]
            cost_of_revenue = financials.loc['Cost Of Revenue'].iloc[0]
            gross_profit = total_revenue - cost_of_revenue
            gross_margin_calc = (gross_profit / total_revenue) * 100
            return round(gross_margin_calc, 2)
        except:
            # Some companies might not have Cost Of Revenue through yFinance
            return 0

    return 0

def get_all_gross_profit_margins(tickers):
    measures = {}
    for ticker in tickers:
        measures[ticker] = get_gross_profit_margins(ticker)  # Fixed: was calling itself
    return measures

# Get Buyback Values
def get_buyback_value(ticker):
    stock = yf.Ticker(ticker)
    cashflow = stock.cashflow

    # Check if cashflow data exists
    if cashflow.empty:
        return 0

    # Buyback value has different names in yFinance - trying different ones
    possible_names = [
        'Repurchase Of Capital Stock',
        'Repurchase of Capital Stock',
        'Share Repurchases',
        'Purchase Of Treasury Stock',
        'Repurchase of Stock',
        'Stock Repurchases'
    ]

    for name in possible_names:
        if name in cashflow.index:
            buyback_cash_flow = abs(cashflow.loc[name].iloc[0])
            buyback_value_billions = buyback_cash_flow / 1e9
            return round(buyback_value_billions, 2)

    # If no buyback data found
    return 0

def get_all_buyback_values(tickers):
    metrics = {}
    for ticker in tickers:
        metrics[ticker] = get_buyback_value(ticker)
    return metrics

# Combine into one DF
def create_combined_metrics_df(tickers):
    # Get all metrics
    sharpe_data = all_sharpe_ratios(tickers)
    market_cap_data = get_all_eoy_market_caps(tickers)
    revenue_growth_data = get_all_revenue_growth(tickers)
    pxr_data = get_all_pxr(tickers)
    gross_margin_data = get_all_gross_profit_margins(tickers)
    buyback_data = get_all_buyback_values(tickers)

    # Create DataFrame
    df = pd.DataFrame({
        'Ticker': tickers,
        'Sharpe Ratio': [sharpe_data[ticker] for ticker in tickers],
        '2024 Market Cap (B)': [market_cap_data[ticker] for ticker in tickers],
        'Revenue Growth %': [revenue_growth_data[ticker] for ticker in tickers],
        'Price X Revenue': [pxr_data[ticker] for ticker in tickers],
        'Gross Profit Margins %': [gross_margin_data[ticker] for ticker in tickers],
        'Buyback Value (B)': [buyback_data[ticker] for ticker in tickers]
    })

    return df

# CREATE THE COMBINED DATAFRAME
combined_df = create_combined_metrics_df(tickers)
print(combined_df)

HTTP Error 401: 
HTTP Error 401: 
HTTP Error 401: 
HTTP Error 401: 


   Ticker  Sharpe Ratio  2024 Market Cap (B)  Revenue Growth %  \
0    META         0.950              1269.12             16.10   
1    NICE         0.055                10.74              6.20   
2    KLAC         0.261                82.94             29.80   
3    REGN        -1.832                75.40             -3.70   
4    MITK        -0.227                 0.51             10.60   
5    AMAT        -0.510               129.84              6.80   
6     FIS         0.250                41.99              2.60   
7    COUR         0.581                 1.37              6.00   
8    GILD         1.756               113.30             -0.30   
9    GOOG        -0.193              1037.18             12.00   
10    UIS         0.312                 0.45            -11.40   
11    AMD        -0.249               195.85             13.69   
12    ROK         0.713                31.93             -5.90   
13   DELL        -0.141                38.74              5.10   
14   GNRC 